# 5.3.2 NGSolve - PETSc interface
We use the ngs2petsc interface to map vectors and matrices between NGSolve and PETSc

'/Users/joachim/gitlab/ngsolve/docs/i-tutorials/unit-5a.3-petsc'

In [31]:
from os import getcwd
import imp
imp.load_source("ngs2petsc", getcwd()+"/ngs2petsc.py")

<module 'ngs2petsc' from '/Users/joachim/gitlab/ngsolve/docs/i-tutorials/unit-5a.3-petsc/ngs2petsc.py'>

In [18]:
from ipyparallel import Client
c = Client(profile='mpi')
c.ids

[0, 1, 2, 3]

In [19]:
%%px
from ngsolve import *
from netgen.geom2d import unit_square
comm = MPI.COMM_WORLD
if comm.rank == 0:
    ngmesh = unit_square.GenerateMesh(maxh=0.1).Distribute(comm)
else:
    ngmesh = netgen.meshing.Mesh.Receive(comm)
    
# for l in range(4):
#    ngmesh.Refine()
mesh = Mesh(ngmesh)
from os import getcwd
getcwd()

Out[0:2]: '/Users/joachim'

Out[1:2]: '/Users/joachim'

Out[2:2]: '/Users/joachim'

Out[3:2]: '/Users/joachim'

The Python-module [ngs2petsc.py](ngs2petsc.py) provides functionality to transfer vectors and matrices between NGSolve and Python. 

Make sure that the ipyparallel server can import the module, e.g. by starting the cluster in the current directory.

In [ ]:
%%px
import ngs2petsc as n2p
import petsc4py.PETSc as psc

In [ ]:
%%px
fes = H1(mesh, order=1)
u,v = fes.TnT()
a = BilinearForm(grad(u)*grad(v)*dx+u*v*ds).Assemble()
f = LinearForm(x*v*dx).Assemble()
gfu = GridFunction(fes)

The function CreatePETScMatrix takes an NGSolve matrix, and creates a PETSc matrix from it. A VectorMapping object can map vectors between NGSolve and PETSc. 

In [ ]:
%%px
psc_mat = n2p.CreatePETScMatrix(a.mat)
vecmap = n2p.VectorMapping (a.mat.row_pardofs)

In [ ]:
%%px
psc_f = vecmap.N2P(f.vec)
psc_u = vecmap.CreatePETScVector()

ksp = psc.KSP()
ksp.create()
ksp.setOperators(psc_mat)
ksp.setType(psc.KSP.Type.CG)
ksp.setNormType(psc.KSP.NormType.NORM_NATURAL)
ksp.getPC().setType("gamg")
ksp.setTolerances(rtol=1e-6, atol=0, divtol=1e16, max_it=400)

ksp.solve(psc_f, psc_u)   

vecmap.P2N(psc_u, gfu.vec)
pass

In [ ]:
gfu = c[:]["gfu"]
from ngsolve.webgui import Draw

In [ ]:
Draw (gfu[0])